# Calling Model with Executor
This tutorial shows training and evaluating model with `Executor` and `Loader`.
- Training `model`
- Feed `Loader` into `model`

In the tutorial 1 we shows how to inference images via a specific model.
But it seems a little complicated:
- What's the argument of `eval` function?
- What's the data type it returns?

Therefore we provide an easier way to call models, as we named `Executor`.

In [4]:
from VSR.DataLoader import Dataset, Loader
from VSR.Model import get_model

srcnn_builder = get_model('srcnn')
srcnn = srcnn_builder(scale=3, channel=1)
try:
    srcnn.load("srcnn.pth")
except FileNotFoundError:
    pass
data = Dataset('../../Tests/data').include('*.png')
loader = Loader(data, scale=3, threads=8)
# convert to grayscale image
loader.set_color_space('hr', 'L')
loader.set_color_space('lr', 'L')
with srcnn.get_executor(root=None) as t:
    config = t.query_config({})
    print(list(config.keys()))
    config.epochs = 1
    config.steps = 10
    config.batch_shape = [4, 1, 16, 16]
    config.lr = 1e-3
    t.fit([loader, None], config)

2020-02-11 22:45:13,229 INFO: Total params: 57281
2020-02-11 22:45:13,231 INFO: Fitting: [SRCNN]
100%|##########| 10/10 [00:01<00:00,  6.34batch/s, loss=00.00814, image=00.00814]
2020-02-11 22:45:14,837 INFO: Training [SRCNN] finished.


['epochs', 'batch_shape', 'steps', 'val_steps', 'lr', 'lr_schedule', 'memory_limit', 'inference_results_hooks', 'validate_every_n_epoch', 'traced_val', 'ensemble', 'cuda', 'map_location', 'train_loader', 'val_loader', 'epoch', 'avg_meas']
| 2020-02-11 22:45:13 | Epoch: 1/1 | LR: 0.001 |
| Epoch average loss = 0.053390 |
| Epoch average image = 0.053390 |


The method `get_executor(root)` builds an model executor with a working directory.
If root is not set, the executor runs without saving models.

Using `with` statement to construct a running environment for it.

To config the executor, calling `query_config({})` to get the default configuration.

To train the model, call `fit` method. The first argument is a list of two loaders,
the first one is the loader for training and the second loader is for validating (can be None).

To evaluate or inference the model, call `benchmark` or `infer` method.

In [8]:
from VSR.DataLoader import Dataset, Loader
from VSR.Model import get_model

def my_hook(outputs: list, names: list):
    print(len(outputs), outputs[0].shape)
    print(names)

srcnn_builder = get_model('srcnn')
srcnn = srcnn_builder(scale=3, channel=1)
try:
    srcnn.load("srcnn.pth")
except FileNotFoundError:
    pass
data = Dataset('../../Tests/data').include('*.png')
loader = Loader(data, scale=3, threads=8)
# convert to grayscale image
loader.set_color_space('hr', 'L')
loader.set_color_space('lr', 'L')
with srcnn.get_executor(root=None) as t:
    config = t.query_config({})
    config.inference_results_hooks = [my_hook]
    # t.benchmark(loader, config)
    t.infer(loader, config)

2020-02-11 22:46:43,370 INFO: Total params: 57281
2020-02-11 22:46:43,381 INFO: Inferring [srcnn] at epoch 0
Infer: 100%|##########| 16/16 [00:19<00:00,  1.24s/it]


1 (1, 1, 384, 510)
['img0']
1 (1, 1, 384, 510)
['img1']
1 (1, 1, 375, 1242)
['c_10']
1 (1, 1, 375, 1242)
['c_11']
1 (1, 1, 375, 1242)
['f_01']
1 (1, 1, 126, 126)
['img_003_SRF_2_LR']
1 (1, 1, 255, 255)
['img_001_SRF_2_LR']
1 (1, 1, 138, 138)
['img_004_SRF_2_LR']
1 (1, 1, 144, 144)
['img_002_SRF_2_LR']
1 (1, 1, 270, 480)
['xiuxian001']
1 (1, 1, 171, 114)
['img_005_SRF_2_LR']
1 (1, 1, 135, 240)
['xiuxian002']
1 (1, 1, 135, 240)
['xiuxian001']
1 (1, 1, 135, 240)
['xiuxian003']
1 (1, 1, 270, 480)
['xiuxian002']
1 (1, 1, 270, 480)
['xiuxian003']
